# Install and import packages needed for Colab

This algorithm assumes you are using Google Colab, if you are using a different dev environment, some adjustments may be needed

In [ ]:
from google.colab import drive
drive.mount("mnt")

%cd "mnt/My Drive/Colab Notebooks/adversarial_examples/"

!pip install import-ipynb
import import_ipynb

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install git+https://github.com/RobustBench/robustbench.git

In [ ]:
! pip install pymoo
! pip install pytorch_lightning
! pip install --upgrade pytorch-lightning

In [ ]:
from google.colab import drive
drive.mount("mnt")

%cd "mnt/My Drive/Colab Notebooks/adversarial_examples/"

!pip install import-ipynb
import import_ipynb

In [ ]:
%run Cifar10Models.ipynb

In [ ]:
from MOAA import *
from LossFunctions import *
from Solutions import *
from operators import *
# from ImageNetModels import *
import time
import numpy as np
import argparse
import os
import torch
# from robustbench.data import load_cifar10, load_imagenet
from robustbench.data import *
# from robustbench.data import load_imagenet
from operator import attrgetter
# from _typeshed import IdentityFunction
from Solutions import __tournament

# Parameters & Attack (Method)

In [ ]:
#Parameters
pc_pixels = 0.1
pm_pixels = 0.4
m_eta = 40
c_eta = 10
pop_size = 100
eps = 1024

# The model won't correctly pick 100% of the images. Pick a number higher than 100 to account for this
experiment_size = 106

x_test, y_test = load_cifar10(n_examples = experiment_size)
model = Cifar10Model()

Let's verify that the images are correctly classified by the model in the first place

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
verify_class = np.empty(0)
verify_pred =np.empty(0)
for i in range(experiment_size):
  verify_class = np.append(verify_class,y_test[i])

  img_ = x_test[i]
  img_ = img_[None, :]
  if device == 'cuda':
    img_ = img_.to('cuda')

  preds = model.predict(img_).flatten()
  y = int(torch.argmax(preds))
  verify_pred = np.append(verify_pred,y)

In [ ]:
np.sum(verify_class==verify_pred)

In [ ]:
experiment_images = torch.empty(0,3,32,32).to(device)
experiment_classes = []
for i in range(len(x_test)):
  img_ = x_test[i]
  img_ = img_[None, :]
  if device == 'cuda':
    img_ = img_.to('cuda')

  preds = model.predict(img_).flatten()
  y = int(torch.argmax(preds))

  if y_test[i] == y:
    experiment_images = torch.cat((experiment_images,img_),dim=0)
    experiment_classes.append(y)
experiment_classes = torch.tensor(experiment_classes)

In [ ]:
experiment_images = experiment_images.cpu()
experiment_classes = experiment_classes.cpu()
len(experiment_images)

The main attack loop for the experiment

In [ ]:
for i in range(len(experiment_images)):
  np.random.seed(0)
  random.seed(0)
  print(i)
  loss = UnTargeted(model, experiment_classes[i].numpy(), to_pytorch=True) # to_pytorch is True only is the model is a pytorch model
  params = {
      "i": i,
      "x": experiment_images[i].permute(1, 2, 0).numpy(), # Image is assume to be numpy array of shape height * width * 3
      "eps": eps, # set to the maximum number of pixels in the image
      "init_pixels": [0.025,0.05,0.10,0.20,0.40,0.80], # number of changed pixels
      "init_p_size": np.linspace(start=0.08,stop=0.80,num=20), # Perturbation values have {-p_size, p_size, 0}. Change this if you want smaller perturbations.
      "iterations": 10000 // pop_size, # model query budget / population size
      "pc_pixels": pc_pixels, # crossover for swapping pixels
      "pm_pixels": pm_pixels, # mutation parameter for changing pixel locations
      "m_eta": m_eta, # eta parameter for polynomial mutation
      "c_eta": c_eta, # eta parameter for sbx crossover
      "pop_size": pop_size, # population size
      "zero_probability": 0.3,
      "tournament_size": 2 # Number of parents compared to generate new solutions, cannot be larger than the population
  }
  print("Starting attack")
  attack = Attack(params)
  attack.attack(loss)